In [12]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [13]:
LOCAL_PATH = 'data'
RAW_TRACE = 'Skype_HongKong.pcapng'
TCP_TRACE_FEATURE_FILE = 'Skype_HongKong_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'Skype_HongKong_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'Skype_HongKong_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'Skype_HongKong_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [14]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [15]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 6.09 s, sys: 489 ms, total: 6.58 s
Wall time: 47.4 s
Conversion done
CPU times: user 312 ms, sys: 39.4 ms, total: 351 ms
Wall time: 28.8 s


In [18]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow):
    extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow)[features].to_csv(file, index=False)

In [21]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=None)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=None)
    else:
        raise

1.0 samp rate with upsampling: 100%|██████████| 211/211 [00:02<00:00, 87.49it/s] 
